<a id="installation"></a>
# Installation

In [2]:
!pip install -r scripts/requirements.txt

# Modules
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import requests
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np
from lets_plot import *
from palmerpenguins import load_penguins
from urllib import request
from io import BytesIO
from sklearn.cluster import KMeans

# Fonctions
from scripts import import_eec
from scripts import import_eec_all



/opt/python/lib/python3.13/site-packages/palmerpenguins/penguins.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Import des données détail de l'enquête Emploi de 2010 à 2024

Je n'ai pas trouvé les données dans la base Melodi et l'Insee mais directement en ligne sur le site. J'ai essayé d'utiliser la fonction `pd.read_csv()` pour les télécharger directement depuis l'URL, mais les données sont souvent compressées au format .zip avec un autre fichier de documentation. J'ai rédigé une fonction `pd.read_from_zip()` qui importe le plus gros fichier .csv contenu dans le dossier zippé au bout de l'URL. Puis, pour les données antérieures à 2018, j'ai adapté la fonction pour pouvoir lire également les fichiers `dbase` car le format `CSV` n'est pas disponible. J'ai aussi ajouté fait une copie des données sur le cloud de mon compte SSPCloud et mis un lien "backup" qu'on peut indiquer à la fonction si le lien direct ne fonctionne pas (j'ai vu que c'était tout le temps fait comme ça dans le cours).

## Assemblage des tables des différentes années

Les variables présentes dans les différentes tables ne sont pas les mêmes d'année en année. Par exemple, la variable d'âge est en 5 modalités et s'appelle `AGE5` en 2018 et 2019 puis elle passe à 6 modalités avec un découpage différent à partir de 2020 et s'appelle alors `AGE6`. Dans un premier temps, on peut ne conserver que les variables qui sont présentes dans toutes les tables, en espérant que les modalités restent les mêmes la plupart du temps. Si on veut en utiliser d'autres, il faudra ajouter des recodages préalables. Pour simplifier les choses, j'ai conservé dans un premier temps uniquement les variables présentes dans toutes les tables. Il y en a 22 qui ont le même nom entre 2010 et 2024. Pour ne pas s'encombrer avec des variables qu'on utilise pas, on peut faire une liste des variables qu'on veut utiliser parmi celles-ci, qu'on peut allonger selon les besoins.

## Sauvegarde intermédiaire

Pour éviter d'avoir à faire systématiquement cette phase d'importation qui est longue (en particulier pour les fichier .dbf), j'ai exporté la base obtenue (avec les 22 variables) dans un fichier `PARQUET` que l'on peut importer directement.

In [ ]:
# Import depuis le site de l'Insee en utilisant les fonctions des scripts d'import
# from import_eec_all import load_eec_all

# core_vars = ['AAC', 'ACTEU', 'ANCCHOM', 'ANNEE', 'CHPUB', 'DISPPLC', 
#             'EXTRIAN', 'OFFICC', 'PASTRA', 'PASTRB', 'PUB3FP', 'RABS', 
#             'RAISTP', 'SEXE', 'SOUSEMPL', 'STATUT', 'STC', 'STPLC', 
#             'TEMP', 'TPPRED', 'TRAREF', 'TRIM']

# eec_all = load_eec_all(core_vars=core_vars)

# Sauvegarde
# eec_all.to_parquet("data/eec_all.parquet", index=False)


In [3]:
# Import direct du fichier `PARQUET`
eec_all = pd.read_parquet("https://minio.lab.sspcloud.fr/phobeika/open_eec/eec_all.parquet")


In [ ]:
# En fait on ne garde que les actifs en emploi (`ACTEU` == 1)
eec_actifs = eec_all[eec_all['ACTEU'] == "1"]
eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)

/tmp/ipykernel_29398/1482212354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)


In [ ]:
LetsPlot.setup_html()

In [ ]:
eec_actifs

,AAC,ACTEU,ANCCHOM,ANNEE,CHPUB,DISPPLC,EXTRIAN,OFFICC,PASTRA,PASTRB,...,RAISTP,SEXE,SOUSEMPL,STATUT,STC,STPLC,TEMP,TPPRED,TRAREF,TRIM
3,,1,,2010,2,,154.013793,3,,,...,,1,,45,2,2,,1,1,1
4,,1,,2010,2,,148.713215,3,,,...,,1,,45,2,2,,1,1,2
5,,1,,2010,2,,160.098185,3,1,,...,,1,,45,2,2,,1,2,3
6,,1,,2010,2,,150.064252,3,,,...,,1,,45,2,2,,1,1,4
8,1,1,,2010,3,,157.836818,1,2,2,...,,1,,43,2,2,,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5933196,<NA>,1,<NA>,2024,7.0,<NA>,54.183092,2.0,2.0,9.0,...,<NA>,1,<NA>,2.0,3.0,2.0,<NA>,1.0,2.0,4
5933197,<NA>,1,<NA>,2024,1.0,1.0,41.192897,2.0,<NA>,<NA>,...,<NA>,2,<NA>,2.0,3.0,1.0,<NA>,1.0,1.0,4
5933200,<NA>,1,<NA>,2024,1.0,<NA>,42.420376,2.0,<NA>,<NA>,...,<NA>,1,<NA>,2.0,3.0,2.0,<NA>,1.0,1.0,4
5933203,<NA>,1,<NA>,2024,7.0,1.0,63.340005,2.0,2.0,9.0,...,9.0,1,1.0,2.0,3.0,1.0,<NA>,2.0,2.0,4


In [ ]:
ggplot(eec_actifs, aes(x = 'ANNEE')) + geom_bar()